#    
<h1></h1>
<h1><center>Bienvenido al Segundo Parcial, USUARIO!</center></h1>


Se usara el conjunto de datos "Book Genome Dataset", correspondiente a un conjunto de datos que contiene puntuaciones que indican el grado en que las etiquetas se aplican a elementos, como películas o libros. En especifico los datos raw constan de un subconjunto de datos de Goodreads [Wan y McAuley, 2018, Wan et al., 2019] y calificaciones de etiquetas de libros. El subconjunto de Goodreads incluye información sobre libros populares, como títulos, autores, años de lanzamiento, valoraciones de usuarios, reseñas y estanterías. Descargue el archivo "book_genoma.zip" de la pestaña DataSet, considere solo los archivos "ratings.json" y "metadata.json", en la pestaña "descripción" encontrara información de cada uno de estos archivos. Una vez descomprima el archivo .zip la ubicacion de los archivos es: archivos en:
- .\book_dataset\raw\ratings.json
- .\book_dataset\raw\metadata.json

In [1]:
! pip install findspark pyspark

###Punto 1

(10%) Inicialice Spark y cargue los datos al contexto, para ello:

1. Cree un jupyter notebook y cree un SparkContext que se llame sc.

2. Cargue los datos del archivo "ratings.json" a una RDD llamado "ratings".
3. Usando el archivo metadata.json cree un diccionario llamado "titulos", como variable broadcast de spark, que contenga como "key" el id del libro (id_item) y como "valor" una lista con los siguientes elementos: El titulo (title), los autores (authors), y el año

In [2]:
from google.colab import drive
drive.mount('/content/drive')

from pyspark import SparkConf, SparkContext
import findspark
import json

findspark.init()
conf = SparkConf().setMaster("local[*]").setAppName("Parcial2")
sc = SparkContext(conf = conf)

Mounted at /content/drive


In [3]:
ratings = sc.textFile('/content/drive/MyDrive/Semestre 9/Arquitectura BigData/Seguimiento 2/book-data/ratings.json')

def parseline(x):
    line = json.loads(x)
    return (str(line['item_id']), float(line['rating']))

ratings = ratings.map(parseline)

In [4]:
def built_metadata():
  dictio = {}
  with open('/content/drive/MyDrive/Semestre 9/Arquitectura BigData/Seguimiento 2/book-data/metadata.json') as file:
    for line in file.readlines():
      line = json.loads(line)
      dictio[str(line['item_id'])] = {
          'title' : line['title'],
          'year' :  line['year'],
          'authors': line['authors']
      }
  return dictio

titulos = sc.broadcast(built_metadata())

###Punto 2

(20%) Liste el (los) nombre(s) de(l) autor(es) del libro con mejor rating, en caso de ser más de un libro listelos todos.

In [5]:
ratings_prom = ratings.mapValues(lambda x: (x, 1))\
                      .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                      .mapValues(lambda x: round(x[0] / x[1], 2))\
                      .sortBy(lambda x: -x[1])\
                      .map(lambda x: (x[0], x[1]))

rating_max = ratings_prom.take(1)[0][1]

results = ratings_prom.filter(lambda x: x[1] == rating_max)\
                          .map(lambda x: {
                              'title': titulos.value[x[0]]['title'],
                              'authors': titulos.value[x[0]]['authors'],
                              'rating': x[1]
                          }).collect()

for book in results:
    print(f"Título: {book['title']} \nAutor(es): {book['authors']}\nRating: {book['rating']}")

Título: Lovely Trigger (Tristan & Danika, #3) 
Autor(es): R.K. Lilley
Rating: 4.83


###Punto 3
(30%) Liste los nombres de los libros con un rating promedio de minimo 4.0 ordenados de mayor a menor rating. Para ello:
1. (10%) Usando el RDD "ratings" calcule el rating promedio para cada libro.

2. (10%) Usando el resultado anterior y la variable broadcast creada, liste los nombres de los libros y sus autores para los libros con un rating promedio de minimo 4.0 como rating promedio y ordénelos de mayor a menor.
3. (10%) De los libros anteriores extraiga el año de cada uno y reporte el año que contenga mayor cantidad de obras maestras. Sugerencia, una vez extraiga los años calcule la distribución de estos, es decir, el conteo y quedese con el que más tenga.

In [6]:
ratings_prom = ratings.mapValues(lambda x: (x, 1))\
                      .reduceByKey(lambda x, y: (x[0] + y[0], x[1] + y[1]))\
                      .mapValues(lambda x: round(x[0] / x[1], 2))

filtrolibAu = ratings_prom.filter(lambda x: x[1] >= 4.0)\
                          .sortBy(lambda x: -x[1])

results = filtrolibAu.map(lambda x: {
                                     'title': titulos.value[x[0]]['title'],
                                     'authors': titulos.value[x[0]]['authors'],
                                     'rating': x[1]})\
                      .collect()

for book in results:
    print(f"Rating: {book['rating']} Título: {book['title']} Autor(es): {book['authors']}")

Rating: 4.83 Título: Lovely Trigger (Tristan & Danika, #3) Autor(es): R.K. Lilley
Rating: 4.82 Título: The Gravity of Us (Elements, #4) Autor(es): Brittainy C. Cherry
Rating: 4.8 Título: March: Book Three (March, #3) Autor(es): John             Lewis, Andrew Aydin
Rating: 4.79 Título: Harry Potter Boxset (Harry Potter, #1-7) Autor(es): J.K. Rowling
Rating: 4.79 Título: Long Way Down (Calloway Sisters, #4; Addicted, #3.2) Autor(es): Krista Ritchie, Becca Ritchie
Rating: 4.78 Título: Just Mercy: A Story of Justice and Redemption Autor(es): Bryan Stevenson
Rating: 4.77 Título: Words of Radiance (The Stormlight Archive, #2) Autor(es): Brandon Sanderson
Rating: 4.77 Título: Grip (Grip, #1) Autor(es): Kennedy Ryan
Rating: 4.77 Título: Wildfire (Hidden Legacy, #3) Autor(es): Ilona Andrews
Rating: 4.77 Título: More Than Forever (More Than, #4) Autor(es): Jay McLean
Rating: 4.76 Título: The Emotion Thesaurus: A Writer's Guide to Character Expression Autor(es): Angela Ackerman, Becca Puglisi
Rat

In [7]:
years = filtrolibAu.map(lambda x: titulos.value[x[0]]['year'])

years_dist = years.map(lambda x: (x, 1)) \
                         .reduceByKey(lambda x, y: x + y)

years_moda = years_dist.max(lambda x: x[1])

print(f"El año con más obras con rating >= 4.0, es el {years_moda[0]} con {years_moda[1]} libros")

El año con más obras con rating >= 4.0, es el 2013 con 548 libros


###Punto 4
(40%) Cree una aplicación en Flask que exponga dos ENDPOINT's que hagan lo siguiente:
1. (20%) Cree un ENDPOINT que se llame “book_by_year” que acepte un parámetro llamado “year” y devuelva una lista con los nombres, autores y la calificación de los mejores 10 libros de ese año. Tenga en cuenta que para este ENDPOINT los promedios de los libros deben de ser calculados considerando solo los libros filtrados por el valor del parámetro year.
2. (20%) Cree un ENDPOINT que se llame "get_books_by_author" que acepte un parámetro llamado “author” y devuelva una lista con todos los titulos de libros y los años asociados al autor. Cambie los espacios en blanco por "+" para el valor del autor, es decir si el autor se llama "John Green" el valor del parametro sera "John+Green", o "john+green", o "JHON+GREEN"; en los tres casos deberia funcionar y devolver exactamente la misa lista, los libros y años del autor "John Green"

In [8]:
#ngrok
! pip install pyngrok
! ngrok authtoken 2oILEUkg8FP4qNHVEzyLVbnEU0M_6e3GutVah3589eqGbuxLQ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from flask import Flask, request, jsonify
from pyngrok import ngrok

In [10]:
app = Flask('Book Ratings')

@app.route('/')
def index():
    return '<h2>Bienvenido a la biblioteca</h2>'

# /book_by_year
# /book_by_year?year=2.002
@app.route('/book_by_year')
def book_by_year():
    year = request.args.get('year')
    if year is None:
        return 'Especifique un año en el parámetro “year”, eg: year=2005', 400
    try:
        cleaned_year = int(year.replace('.', ''))
    except (ValueError, TypeError):
        return 'El parámetro "year" debe ser un número entero válido.', 400

    books_by_year = ratings_prom.filter(lambda x: int(titulos.value[x[0]]['year']) == cleaned_year) \
                                .map(lambda x: (x[0], x[1])) \
                                .sortBy(lambda x: -x[1]) \
                                .take(10)

    results = [{
        'title': titulos.value[book_id]['title'],
        'authors': titulos.value[book_id]['authors'],
        'rating': rating
    } for book_id, rating in books_by_year]

    return jsonify(results)

# /get_books_by_author?author=fulanito+de+tal
# /get_books_by_author?author=John+Green
@app.route('/get_books_by_author')
def get_books_by_author():

    author = request.args.get('author')
    if not author:
        return "El parámetro 'author' es requerido.", 400

    author = author.replace('+', ' ').lower()

    results = ratings_prom.filter(lambda x: any(author == a.lower() for a in titulos.value[x[0]]['authors'].split(', '))) \
                          .map(lambda x: {
                              'title': titulos.value[x[0]]['title'],
                              'year': titulos.value[x[0]]['year']
                          }) \
                          .collect()

    if not results:
        return f"No se encontraron libros del autor '{author.replace(' ', '+')}'.", 404

    return jsonify(results)

tunnel_flask = ngrok.connect(5000)
print(f'URL ngrok for Flask: {tunnel_flask.public_url}')

app.run()

URL ngrok for Flask: https://d2f0-34-125-166-95.ngrok-free.app
 * Serving Flask app 'Book Ratings'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 18:47:36] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 18:47:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 18:47:45] "GET /book_by_year?year=2.002 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [09/Nov/2024 18:48:02] "GET /get_books_by_author?author=John+Green HTTP/1.1" 200 -
